In [1]:
# implement the stochastic gradient descent learning algorithm for a FNN.
# Gradients are calculated using backpropagation.

import random
import numpy as np

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
 
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def train(self, training_data_size, epochs, mini_batch_size, eta, lmbda, evaluation_data_size,
              monitor_training_cost=True,
              monitor_training_accuracy=True,
              monitor_evaluation_cost=True,
              monitor_evaluation_accuracy=True):

        training_cost, training_accuracy = [], []
        evaluation_cost, evaluation_accuracy = [], []
        
        evaluation_data = mnist.test.next_batch(evaluation_data_size, fake_data=False)
        evaluation_data = [(np.transpose([x]), np.transpose([y])) 
                           for x, y in zip(evaluation_data[0], evaluation_data[1])]
        
        for j in range(epochs):
            for i in range(0, training_data_size, mini_batch_size):
                training_data = mnist.train.next_batch(mini_batch_size, fake_data=False)
                training_data = [(np.transpose([x]), np.transpose([y]))
                                 for x, y in zip(training_data[0], training_data[1])]
                self.train_mini_batch(training_data, eta, lmbda, training_data_size)
        
            print("Epoch %s training complete" % j)

            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
                
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, mini_batch_size))
                
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
                
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(accuracy,
                                                                    evaluation_data_size))

        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

    def train_mini_batch(self, mini_batch, eta, lmbda, n):
        '''Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        '''
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [(1 - eta * (lmbda / n)) * w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        '''Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``.
        '''
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        # backward pass
        delta = self.cost_delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())

        return (nabla_b, nabla_w)

    def cost(self, a, y):
        return np.sum(np.nan_to_num(-y * np.log(a) - (1 - y) * np.log(1 - a)))
    
    def cost_delta(self, z, a, y):
        return a - y
    
    def total_cost(self, data, lmbda):
        '''Return the total cost for the data set ``data``.'''
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            cost += self.cost(a, y) / len(data)
            cost += 0.5 * (lmbda / len(data)) * sum(np.linalg.norm(w) ** 2
                                                    for w in self.weights)
        return cost
    
    def accuracy(self, data):
        '''Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.
        '''
        results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                   for (x, y) in data]

        result_accuracy = sum(int(x == y) for (x, y) in results)
        return result_accuracy

In [14]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [16]:
net = Network([784, 30, 10])
net.train(50000, 30, 10, 0.3, 0.5, 10000)

Epoch 0 training complete
Cost on training data: 5857.6303135547405
Accuracy on training data: 9 / 10
Cost on evaluation data: 5857.590773441423
Accuracy on evaluation data: 9063 / 10000
Epoch 1 training complete
Cost on training data: 5755.524626713724
Accuracy on training data: 9 / 10
Cost on evaluation data: 5755.625013026632
Accuracy on evaluation data: 9225 / 10000
Epoch 2 training complete
Cost on training data: 5645.450276604893
Accuracy on training data: 10 / 10
Cost on evaluation data: 5645.792908134097
Accuracy on evaluation data: 9313 / 10000
Epoch 3 training complete
Cost on training data: 5540.591674595329
Accuracy on training data: 10 / 10
Cost on evaluation data: 5540.940773963129
Accuracy on evaluation data: 9336 / 10000
Epoch 4 training complete
Cost on training data: 5437.453651747849
Accuracy on training data: 10 / 10
Cost on evaluation data: 5437.669930291051
Accuracy on evaluation data: 9379 / 10000
Epoch 5 training complete
Cost on training data: 5335.422805668291

([5857.5907734414232,
  5755.6250130266317,
  5645.7929081340972,
  5540.9407739631288,
  5437.6699302910511,
  5335.4738349746121,
  5235.1745827034174,
  5138.5952302731785,
  5042.871859116317,
  4954.7447515268505,
  4865.7816274679035,
  4782.7139326410979,
  4703.3548528287192,
  4628.1594771902064,
  4550.84097658479,
  4474.977440744241,
  4409.1027745609217,
  4343.0794610817129,
  4279.3517813220669,
  4218.6759376386408,
  4160.5448971807909,
  4102.11164515174,
  4047.5177497627674,
  3996.5312795708578,
  3947.0870722462537,
  3898.16580596521,
  3854.1467722267712,
  3807.4999793964776,
  3765.1703440469078,
  3723.8990500319846],
 [9063,
  9225,
  9313,
  9336,
  9379,
  9408,
  9424,
  9442,
  9461,
  9473,
  9498,
  9487,
  9469,
  9528,
  9516,
  9516,
  9524,
  9541,
  9528,
  9550,
  9532,
  9546,
  9567,
  9532,
  9566,
  9575,
  9550,
  9542,
  9560,
  9554],
 [5857.6303135547405,
  5755.5246267137236,
  5645.4502766048927,
  5540.5916745953291,
  5437.45365174784